# Models!

### Importing github repository

In [1]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
token = user_secrets.get_secret("token")

!git clone https://{token}@github.com/Benedart/RecSys-2022-Challenge-Polimi.git

Cloning into 'RecSys-2022-Challenge-Polimi'...
remote: Enumerating objects: 648, done.
remote: Counting objects: 100% (33/33), done.
remote: Compressing objects: 100% (25/25), done.
remote: Total 648 (delta 15), reused 24 (delta 8), pack-reused 615
Receiving objects: 100% (648/648), 1.08 GiB | 27.53 MiB/s, done.
Resolving deltas: 100% (281/281), done.
Updating files: 100% (205/205), done.


#### Compiling Cython files

In [2]:
import os

os.chdir( "./RecSys-2022-Challenge-Polimi")
!python run_compile_all_cython.py

run_compile_all_cython: Found 10 Cython files in 4 folders...
run_compile_all_cython: All files will be compiled using your current python environment: '/opt/conda/bin/python'
Compiling [1/10]: MatrixFactorization_Cython_Epoch.pyx... 
In file included from /opt/conda/lib/python3.7/site-packages/numpy/core/include/numpy/ndarraytypes.h:1969,
                 from /opt/conda/lib/python3.7/site-packages/numpy/core/include/numpy/ndarrayobject.h:12,
                 from /opt/conda/lib/python3.7/site-packages/numpy/core/include/numpy/arrayobject.h:4,
                 from MatrixFactorization_Cython_Epoch.c:746:
/opt/conda/lib/python3.7/site-packages/numpy/core/include/numpy/npy_1_7_deprecated_api.h:17:2: warning: #warning "Using deprecated NumPy API, disable it with " "#define NPY_NO_DEPRECATED_API NPY_1_7_API_VERSION" [-Wcpp]
   17 | #warning "Using deprecated NumPy API, disable it with " \
      |  ^~~~~~~
MatrixFactorization_Cython_Epoch.c: In function ‘__pyx_pf_32MatrixFactorization_Cyth

In [3]:
!rm -r models/*

In [4]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import scipy.sparse as sps
import csv

from Recommenders.Recommender_import_list import *

# Data preprocessing
#### Importing URM & ICM

In [5]:
URM_all = pd.read_csv("/kaggle/input/recommender-system-2022-challenge-polimi/interactions_and_impressions.csv")
ICM_type = pd.read_csv("/kaggle/input/recommender-system-2022-challenge-polimi/data_ICM_type.csv")
ICM_length = pd.read_csv("/kaggle/input/recommender-system-2022-challenge-polimi/data_ICM_length.csv")

ICM_all = pd.concat([ICM_type, ICM_length]).sort_values(by='item_id')

/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3552: DtypeWarning: Columns (2) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [6]:
URM_all

,UserID,ItemID,Impressions,Data
0,0,11,"0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19",1
1,0,21,NaN,0
2,0,21,NaN,0
3,0,21,"20,21,22,23,24,25,26,27,28,29",0
4,0,21,NaN,1
...,...,...,...,...
5826501,41628,20448,NaN,0
5826502,41628,20896,NaN,1
5826503,41628,21506,NaN,1
5826504,41628,22882,NaN,0


In [7]:
ICM_all

,item_id,feature_id,data
0,0,1,1
0,0,0,1
1,1,0,1
1,1,3,1
2,2,4,1
...,...,...,...
23088,27965,1,1
23089,27966,0,1
23089,27966,1,1
23090,27967,4,1


#### Getting rid of duplicates and empty indices
Let's start with the indices, the mapping is done for consistency

In [8]:
users_URM = pd.DataFrame({"UserID": URM_all["UserID"].unique()})
items_URM = pd.DataFrame({"item_id": URM_all["ItemID"].unique()})

all_items_ICM = pd.DataFrame({"item_id": ICM_all["item_id"].unique()})
relevant_ICM_items = items_URM.merge(all_items_ICM, on="item_id")
relevant_ICM = relevant_ICM_items.merge(ICM_all, on="item_id", how="left")

mapped_id, original_id = pd.factorize(URM_all["UserID"].unique())
user_original_ID_to_index = pd.Series(mapped_id, index=original_id)
print("Unique UserID in the URM are {}".format(len(original_id)))

mapped_id, original_id = pd.factorize(URM_all["ItemID"].unique())
item_original_ID_to_index = pd.Series(mapped_id, index=original_id)

feature_original_ID_to_index = pd.Series({
    0: 5,
    1: 0,
    2: 2,
    3: 3,
    7: 1,
    4: 4
}, index = [0, 1, 2, 3, 4, 7]) # feature id mapping

print("Unique FeatureID in the URM are {}".format(len(feature_original_ID_to_index)))

URM_all["UserID"] = URM_all["UserID"].map(user_original_ID_to_index)
URM_all["ItemID"] = URM_all["ItemID"].map(item_original_ID_to_index)
relevant_ICM["item_id"] = relevant_ICM["item_id"].map(item_original_ID_to_index)
relevant_ICM["feature_id"] = relevant_ICM["feature_id"].map(feature_original_ID_to_index)

ICM_all = relevant_ICM

Unique UserID in the URM are 41629
Unique FeatureID in the URM are 6


In [9]:
def subtract_dfs(df1, df2):
    result = pd.merge(df1, df2, indicator=True, how='outer').query('_merge=="left_only"').drop('_merge', axis=1)
    return result

def get_bonus_fakemalus_realmalus(u):
    DF_Clicked = u[u["Data"] == 1].drop(["Impressions"], axis=1)
    DF_Clicked = DF_Clicked.groupby(['UserID', 'ItemID'], as_index=False).count()
    DF_Clicked.rename(inplace=True, columns={"Data": "Clicked"})
    DF_Watched = u[u["Data"] == 0].drop(["Impressions"], axis=1)
    DF_Watched = DF_Watched.groupby(['UserID', 'ItemID'], as_index=False).count()
    DF_Watched.rename(inplace=True, columns={"Data": "Watched"})
    
    # find bonus: interactiors that are both watched and clicked
    comparison_df = DF_Watched.merge(DF_Clicked, on=["UserID", "ItemID"])
    comparison_df = comparison_df.drop(["Clicked", "Watched"], axis=1)
    bonus = np.array((comparison_df.UserID, comparison_df.ItemID))
    
    # fake malus: interactions that have been only watched
    Watched_once_df = DF_Watched[DF_Watched["Watched"]==1]
    Fake_Malus_df = subtract_dfs(Watched_once_df.drop(["Watched"], axis=1), DF_Clicked.drop(["Clicked"], axis=1))
    fake_malus = np.array((Fake_Malus_df.UserID, Fake_Malus_df.ItemID))
    
    # real malus: interactions that have been only clicked
    Clicked_once_df = DF_Clicked[DF_Clicked["Clicked"]==1]
    Real_Malus_df = subtract_dfs(Clicked_once_df.drop(["Clicked"], axis=1), DF_Watched.drop(["Watched"], axis=1))
    real_malus = np.array((Real_Malus_df.UserID, Real_Malus_df.ItemID))

    return bonus, fake_malus, real_malus

def get_modified_urm(urm_original, bonus_amount, bonus_interactions):     # malus negativi!!!!
    bonus_length = bonus_interactions.shape[1]
       
        
    urm_copy = urm_original.copy()
    
    for i in range(0, bonus_length):
        x = urm_copy[bonus_interactions[0][i], bonus_interactions[1][i]]
        if (x!=0): urm_copy[bonus_interactions[0][i], bonus_interactions[1][i]] += bonus_amount
            
    
    return urm_copy

In [10]:
URM_all_binary = URM_all.drop(["Impressions", "Data"], axis=1).drop_duplicates()
URM_all_binary["Rating"]=1.0

ICM_all["feature_id"].unique()

array([5, 0, 4, 2, 3, 1])

In [11]:
n_users = len(user_original_ID_to_index)
n_items = len(item_original_ID_to_index)
n_features = len(feature_original_ID_to_index)

print("Number of users is {}".format(n_users))
print("Number of items is {}".format(n_items))
print("Number of features is {}".format(n_features))

from sklearn.model_selection import train_test_split
seed = 42

(user_ids_train_validation, user_ids_test,
 item_ids_train_validation, item_ids_test,
 ratings_train_validation, ratings_test) = train_test_split(URM_all_binary.UserID,
                                                        URM_all_binary.ItemID,
                                                        URM_all_binary.Rating,
                                                        test_size=0.15,
                                                        shuffle=True,
                                                        random_state=seed)

(user_ids_train, user_ids_validation,
 item_ids_train, item_ids_validation,
 ratings_train, ratings_validation) = train_test_split(user_ids_train_validation,
                                                        item_ids_train_validation,
                                                        ratings_train_validation,
                                                        test_size=0.15,
                                                        shuffle=True,
                                                        random_state=seed)

urm_train = sps.csr_matrix((ratings_train, (user_ids_train, item_ids_train)),
                           shape = (n_users, n_items))

urm_validation = sps.csr_matrix((ratings_validation, (user_ids_validation, item_ids_validation)),
                           shape = (n_users, n_items))

urm_train_validation = sps.csr_matrix((ratings_train_validation, (user_ids_train_validation, item_ids_train_validation)),
                           shape = (n_users, n_items))

urm_test = sps.csr_matrix((ratings_test, (user_ids_test, item_ids_test)),
                           shape = (n_users, n_items))

urm_all = sps.csr_matrix((URM_all_binary.Rating, (URM_all_binary.UserID, URM_all_binary.ItemID)),
                           shape = (n_users, n_items))

icm_all = sps.csr_matrix((np.ones(len(ICM_all["item_id"].values)), 
                          (ICM_all["item_id"].values, ICM_all["feature_id"].values)),
                        shape = (n_items, 6))

icm_all.data = np.ones_like(icm_all.data)

icm_all = sps.csr_matrix(icm_all)
features_per_item = np.ediff1d(icm_all.indptr)

icm_all = sps.csc_matrix(icm_all)
items_per_feature = np.ediff1d(icm_all.indptr)

icm_all = sps.csr_matrix(icm_all) 

urm_train, urm_validation, urm_all, icm_all

Number of users is 41629
Number of items is 24507
Number of features is 6


(<41629x24507 sparse matrix of type '<class 'numpy.float64'>'
 	with 1123227 stored elements in Compressed Sparse Row format>,
 <41629x24507 sparse matrix of type '<class 'numpy.float64'>'
 	with 198217 stored elements in Compressed Sparse Row format>,
 <41629x24507 sparse matrix of type '<class 'numpy.float64'>'
 	with 1554640 stored elements in Compressed Sparse Row format>,
 <24507x6 sparse matrix of type '<class 'numpy.float64'>'
 	with 39260 stored elements in Compressed Sparse Row format>)

In [12]:
from Evaluation.Evaluator import EvaluatorHoldout

evaluator_validation = EvaluatorHoldout(urm_validation, cutoff_list=[10])
evaluator_test = EvaluatorHoldout(urm_test, cutoff_list=[10])

EvaluatorHoldout: Ignoring 1520 ( 3.7%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 976 ( 2.3%) Users that have less than 1 test interactions


# Models

In [13]:
params = {
    'topK': 2492,
    'l1_ratio': 0.09122038444743771,
    'alpha': 0.00031614482201457075,
}

In [14]:
slim_both_bonus = 0.25
slim_only_watched_once_malus = -0.1
slim_only_clicked_once_malus = -0.1

In [15]:
both_watched_and_clicked, only_watched_once, only_clicked_once = get_bonus_fakemalus_realmalus(URM_all)

slim_urm_all = get_modified_urm(urm_all, slim_both_bonus, both_watched_and_clicked)
slim_urm_all = get_modified_urm(slim_urm_all, slim_only_watched_once_malus, only_watched_once)
slim_urm_all = get_modified_urm(slim_urm_all, slim_only_clicked_once_malus, only_clicked_once)

slim_urm_train_validation = get_modified_urm(urm_train_validation, slim_both_bonus, both_watched_and_clicked)
slim_urm_train_validation = get_modified_urm(slim_urm_train_validation, slim_only_watched_once_malus, only_watched_once)
slim_urm_train_validation = get_modified_urm(slim_urm_train_validation, slim_only_clicked_once_malus, only_clicked_once)

slim_urm_train = get_modified_urm(urm_train, slim_both_bonus, both_watched_and_clicked)
slim_urm_train = get_modified_urm(slim_urm_train, slim_only_watched_once_malus, only_watched_once)
slim_urm_train = get_modified_urm(slim_urm_train, slim_only_clicked_once_malus, only_clicked_once)

In [16]:
ICM_Type = ICM_all[ICM_all["feature_id"] != 5] # togliamo la length
icm_type = sps.csr_matrix((ICM_Type.data, (ICM_Type.feature_id, ICM_Type.item_id)),
                         shape = (5, n_items))

In [17]:
stack_alpha = 0.7896376793234108
stacked_urm_all = sps.vstack([slim_urm_all.multiply(stack_alpha), icm_type.multiply(1 - stack_alpha)]) # stack su quella con bonus
stacked_urm_train_validation = sps.vstack([slim_urm_train_validation.multiply(stack_alpha), icm_type.multiply(1 - stack_alpha)]) # stack su quella con bonus
stacked_urm_train = sps.vstack([slim_urm_train.multiply(stack_alpha), icm_type.multiply(1 - stack_alpha)]) # stack su quella con bonus
stacked_urm_all, stacked_urm_train_validation, stacked_urm_train

(<41634x24507 sparse matrix of type '<class 'numpy.float64'>'
 	with 1574270 stored elements in Compressed Sparse Row format>,
 <41634x24507 sparse matrix of type '<class 'numpy.float64'>'
 	with 1341074 stored elements in Compressed Sparse Row format>,
 <41634x24507 sparse matrix of type '<class 'numpy.float64'>'
 	with 1142857 stored elements in Compressed Sparse Row format>)

In [18]:
SLIM_Elastic_all = SLIMElasticNetRecommender(stacked_urm_all)
SLIM_Elastic_all.fit(**params)

SLIM_Elastic_all.save_model("models", "SLIM_Elastic_all.zip")

SLIMElasticNetRecommender: Processed 3566 (14.6%) in 5.00 min. Items per second: 11.88
SLIMElasticNetRecommender: Processed 7381 (30.1%) in 10.00 min. Items per second: 12.30
SLIMElasticNetRecommender: Processed 11329 (46.2%) in 15.00 min. Items per second: 12.59
SLIMElasticNetRecommender: Processed 15300 (62.4%) in 20.00 min. Items per second: 12.75
SLIMElasticNetRecommender: Processed 19184 (78.3%) in 25.00 min. Items per second: 12.79
SLIMElasticNetRecommender: Processed 23276 (95.0%) in 30.00 min. Items per second: 12.93
SLIMElasticNetRecommender: Processed 24507 (100.0%) in 31.44 min. Items per second: 12.99
SLIMElasticNetRecommender: Saving model in file 'modelsSLIM_Elastic_all.zip'
SLIMElasticNetRecommender: Saving complete


In [19]:
SLIM_Elastic_train_validation = SLIMElasticNetRecommender(stacked_urm_train_validation)
SLIM_Elastic_train_validation.fit(**params)

SLIM_Elastic_train_validation.save_model("models", "SLIM_Elastic_train_validation.zip")

SLIMElasticNetRecommender: Processed 4367 (17.8%) in 5.00 min. Items per second: 14.55
SLIMElasticNetRecommender: Processed 9131 (37.3%) in 10.00 min. Items per second: 15.21
SLIMElasticNetRecommender: Processed 13865 (56.6%) in 15.00 min. Items per second: 15.40
SLIMElasticNetRecommender: Processed 18706 (76.3%) in 20.00 min. Items per second: 15.59
SLIMElasticNetRecommender: Processed 23625 (96.4%) in 25.00 min. Items per second: 15.75
SLIMElasticNetRecommender: Processed 24507 (100.0%) in 25.85 min. Items per second: 15.80
SLIMElasticNetRecommender: Saving model in file 'modelsSLIM_Elastic_train_validation.zip'
SLIMElasticNetRecommender: Saving complete


In [20]:
SLIM_Elastic_train = SLIMElasticNetRecommender(stacked_urm_train)
SLIM_Elastic_train.fit(**params)

SLIM_Elastic_train.save_model("models", "SLIM_Elastic_train.zip")

SLIMElasticNetRecommender: Processed 4996 (20.4%) in 5.00 min. Items per second: 16.65
SLIMElasticNetRecommender: Processed 10372 (42.3%) in 10.00 min. Items per second: 17.28
SLIMElasticNetRecommender: Processed 16336 (66.7%) in 15.00 min. Items per second: 18.15
SLIMElasticNetRecommender: Processed 22510 (91.9%) in 20.00 min. Items per second: 18.76
SLIMElasticNetRecommender: Processed 24507 (100.0%) in 21.59 min. Items per second: 18.92
SLIMElasticNetRecommender: Saving model in file 'modelsSLIM_Elastic_train.zip'
SLIMElasticNetRecommender: Saving complete


In [21]:
!mkdir ../models
!cp models/* ../models
os.chdir("..")
!rm -r RecSys-2022-Challenge-Polimi